In [2]:
import pandas as pd
import numpy as np
import requests

Use beautfiul soup to locate the table header and content

In [3]:
from bs4 import BeautifulSoup
page_address='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_content=requests.get(page_address).text
soup = BeautifulSoup(page_content,'html.parser')

sa=[]

#fetch column names
titlerow = soup.find('tr').text
head_=titlerow.split('\n')
del head_[0]
del head_[-1]

print(head_)

#fetch column content
for rows_ in soup.findAll('tr'):
    s=''
    for columns_ in rows_.findAll('td'):
        s=s+','+columns_.text

    s=s[1:]
    s=s.replace('\n','')
    sp=s.split(',')
    sa.append(sp)
  
#clean up      
del sa[0]
for i in range(293,288,-1):
    del sa[i]
    
print('Extracted from html...')

df = pd.DataFrame.from_records(sa, columns=head_)

['Postcode', 'Borough', 'Neighbourhood']
Extracted from html...


In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [1]:
#get rid of Not assigned Boroughs
df1=df[df.Borough != 'Not assigned']

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

df1.sort_values(by=['Postcode','Borough'],inplace=True)

print('Created DataFrame...')

#rename nighbourhoods that have a not assigned value
for i in df1.index:
    if df1.iloc[i,2]=='Not assigned':
        df1.iloc[i,2]=df1.iloc[i,1]
    
print('Assigned neighborhood names...')

In [11]:
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


Group by Postcode

In [8]:
#combine neighbourhoods with the same postalcode in one row
#find unique postal codes

dummy=df1['Postcode']

dummy.drop_duplicates(inplace=True)
df2=pd.DataFrame(dummy)
df2['Neighbourhood']='';
df2['Borough']='';

df2.reset_index(inplace=True)
df2.drop('index',axis=1,inplace=True)
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

#search for values in df1 and copy neighbourhood names to df2. Match criteris = Postcode
for i in df2.index:
    for j in df1.index:
        if df2.iloc[i,0]==df1.iloc[j,0]:
            df2.iloc[i,1]=df1.iloc[j,1]
            df2.iloc[i,2]=df2.iloc[i,2]+','+df1.iloc[j,2]
            
#get rid of extra ','
for i in df2.index:
    s=df2.iloc[i,2]
    if s[0]==',':
        s=s[1:]
    df2.iloc[i,2]=s

Check the grouping

In [10]:
print(df2.head(10))

  Postcode Neighbourhood                                        Borough
0      M1B   Scarborough                                  Rouge,Malvern
1      M1C   Scarborough           Highland Creek,Rouge Hill,Port Union
2      M1E   Scarborough                Guildwood,Morningside,West Hill
3      M1G   Scarborough                                         Woburn
4      M1H   Scarborough                                      Cedarbrae
5      M1J   Scarborough                            Scarborough Village
6      M1K   Scarborough      East Birchmount Park,Ionview,Kennedy Park
7      M1L   Scarborough                  Clairlea,Golden Mile,Oakridge
8      M1M   Scarborough  Cliffcrest,Cliffside,Scarborough Village West
9      M1N   Scarborough                     Birch Cliff,Cliffside West


Check shape

In [12]:
print(df2.shape)

(103, 3)
